In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [3]:
pip install pgmpy

     |████████████████████████████████| 331 kB 1.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
cancer_model = BayesianModel([('Pollution','Cancer'),
                             ('Smoker','Cancer'),
                             ('Cancer','Xray'),
                             ('Cancer','Dyspnoea')])
print('Bayesian network nodes are:')
print("\t",cancer_model.nodes())
print('Bayesian network edges are:')
print("\t",cancer_model.edges())

cpd_poll = TabularCPD(variable='Pollution',variable_card=2,values=[[0.9],[0.1]])
cpd_smoke = TabularCPD(variable='Smoker',variable_card=2,values=[[0.3],[0.7]])
cpd_cancer = TabularCPD(variable='Cancer',variable_card=2,values=[[0.03,0.05,0.001,0.02],
                                                                 [0.97,0.95,0.999,0.98]],
                       evidence=['Smoker','Pollution'],
                       evidence_card=[2,2])
cpd_xray = TabularCPD(variable='Xray',variable_card=2,values=[[0.9,0.2],[0.1,0.8]],
                     evidence=['Cancer'],evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea',variable_card=2,values=[[0.65,0.3],[0.35,0.7]],
                     evidence=['Cancer'],evidence_card=[2])

Bayesian network nodes are:
	 ['Pollution', 'Cancer', 'Smoker', 'Xray', 'Dyspnoea']
Bayesian network edges are:
	 [('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea'), ('Smoker', 'Cancer')]


In [7]:

cancer_model.add_cpds(cpd_poll,cpd_smoke,cpd_cancer,cpd_xray,cpd_dysp)
print('Model generated by adding cpts(cpds)')
print('Checking correctness of model:',end='')
print(cancer_model.check_model())

Model generated by adding cpts(cpds)
Checking correctness of model:True


In [8]:
print('All local depencies are as follows')
cancer_model.get_independencies()

All local depencies are as follows


(Pollution ⟂ Smoker)
(Pollution ⟂ Dyspnoea, Xray | Cancer)
(Pollution ⟂ Dyspnoea, Xray | Cancer, Smoker)
(Pollution ⟂ Xray | Cancer, Dyspnoea)
(Pollution ⟂ Dyspnoea | Cancer, Xray)
(Pollution ⟂ Xray | Cancer, Dyspnoea, Smoker)
(Pollution ⟂ Dyspnoea | Xray, Cancer, Smoker)
(Smoker ⟂ Pollution)
(Smoker ⟂ Dyspnoea, Xray | Cancer)
(Smoker ⟂ Xray | Cancer, Dyspnoea)
(Smoker ⟂ Dyspnoea, Xray | Cancer, Pollution)
(Smoker ⟂ Dyspnoea | Cancer, Xray)
(Smoker ⟂ Xray | Cancer, Dyspnoea, Pollution)
(Smoker ⟂ Dyspnoea | Cancer, Pollution, Xray)
(Xray ⟂ Dyspnoea, Pollution, Smoker | Cancer)
(Xray ⟂ Pollution, Smoker | Cancer, Dyspnoea)
(Xray ⟂ Dyspnoea, Smoker | Cancer, Pollution)
(Xray ⟂ Dyspnoea, Pollution | Cancer, Smoker)
(Xray ⟂ Smoker | Cancer, Dyspnoea, Pollution)
(Xray ⟂ Pollution | Cancer, Dyspnoea, Smoker)
(Xray ⟂ Dyspnoea | Cancer, Pollution, Smoker)
(Dyspnoea ⟂ Pollution, Smoker, Xray | Cancer)
(Dyspnoea ⟂ Pollution, Xray | Cancer, Smoker)
(Dyspnoea ⟂ Smoker, Xray | Cancer, Pollution)
(Dy

In [9]:
print('Displaying  CPDs')
print(cancer_model.get_cpds('Pollution'))
print(cancer_model.get_cpds('Smoker'))
print(cancer_model.get_cpds('Cancer'))
print(cancer_model.get_cpds('Xray'))
print(cancer_model.get_cpds('Dyspnoea'))

Displaying  CPDs
+--------------+-----+
| Pollution(0) | 0.9 |
+--------------+-----+
| Pollution(1) | 0.1 |
+--------------+-----+
+-----------+-----+
| Smoker(0) | 0.3 |
+-----------+-----+
| Smoker(1) | 0.7 |
+-----------+-----+
+-----------+--------------+--------------+--------------+--------------+
| Smoker    | Smoker(0)    | Smoker(0)    | Smoker(1)    | Smoker(1)    |
+-----------+--------------+--------------+--------------+--------------+
| Pollution | Pollution(0) | Pollution(1) | Pollution(0) | Pollution(1) |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(0) | 0.03         | 0.05         | 0.001        | 0.02         |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(1) | 0.97         | 0.95         | 0.999        | 0.98         |
+-----------+--------------+--------------+--------------+--------------+
+---------+-----------+-----------+
| Cancer  | Cancer(0) | Cancer(1) |
+---------+-----------+-------

In [10]:
cancer_infer=VariableElimination(cancer_model)
print('\n Inferencing with bayesian network')
print("\n Probability of Cancer given smoker")
q=cancer_infer.query(variables=['Cancer'],evidence={'Smoker':1})
print(q)

print("\n Probability of Cancer given smoker,pollution")
q=cancer_infer.query(variables=['Cancer'],evidence={'Smoker':1,'Pollution':1})
print(q)

Eliminating: Xray: 100%|██████████| 3/3 [00:00<00:00, 271.50it/s]

Finding Elimination Order: :   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Finding Elimination Order: : 100%|██████████| 2/2 [00:00<00:00, 196.91it/s]


Eliminating: Xray: 100%|██████████| 2/2 [00:00<00:00, 193.34it/s]A


 Inferencing with bayesian network

 Probability of Cancer given smoker
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0029 |
+-----------+---------------+
| Cancer(1) |        0.9971 |
+-----------+---------------+

 Probability of Cancer given smoker,pollution
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0200 |
+-----------+---------------+
| Cancer(1) |        0.9800 |
+-----------+---------------+
